# Linear regression 

In this exercise you will use linear regression to predict flat prices. One more time, training will be handled via gradient descent. Although, contratry to the first exercise, we will now:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a different loss function.

Let's start with getting the data.

In [18]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2021-04-01 15:54:15--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2021-04-01 15:54:15--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uceb5ee401d2393d45c7a2fd8fb6.dl.dropboxusercontent.com/cd/0/get/BLw16bQG09SmbfOZN7RGuj2TnirrsDcePh_C4wlYNu6UD8eUn73V00RYvW42idVDZ39FwVSFuiMUayETvo9_hU35Q8T1bCcLz27ZbkaowENNEHKa-zXFRrNYdiyG1tTLbtJn2bht15GAUQd26MbJhU3q/file?dl=1# [following]
--2021-04-01 15:54:16--  https://uceb5ee401d2393d45c7a2fd8fb6.dl.dropboxusercontent.com/cd/0/get/BLw16bQG09SmbfOZN7RGuj2TnirrsDcePh_C4wlYNu6UD8eUn73V00RYvW42idVDZ39FwVSFuiMUayETvo

In [19]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


In [20]:
import pandas as pd
train = pd.read_csv('mieszkania.csv')
test = pd.read_csv('mieszkania_test.csv')

total = pd.concat([train,test])

total = pd.get_dummies(total, prefix='dzielnica')

train = total[:len(train)]
test = total[len(train):]

Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [21]:
import numpy as np
def msle(ys, ps):
    assert len(ys) == len(ps)
    #################################
    # TODO: Implement this function #
    #################################
    sum = 0
    for i in range(len(ys)):
      sum += (np.log(1 + ys[i]) - np.log(1 + ps[i])) ** 2 
    return (1/len(ys)) * sum

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [22]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################
n_train = len(train)
n_test = len(test)
train_mean = np.mean(train['cena'])
print(f'Mean price for training set: {train_mean}')
msle_train = msle(train['cena'], np.full(n_train, fill_value=train_mean))
print(f'MSLE for the same constant (training set): {msle_train}')
msle_test = msle(test['cena'], np.full(n_test, fill_value=train_mean))
print(f'MSLE for the same constant (test set): {msle_test}')

Mean price for training set: 507919.49
MSLE for the same constant (training set): 0.3915253538257009
MSLE for the same constant (test set): 0.4284115392580848


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [23]:
#############################################
# TODO: Find this constant and compute msle #
#############################################
c = np.exp(np.mean(np.log(1 + train['cena']))) - 1
msle_const = msle(test['cena'], np.full(n_test, c))
print(f'MSLE for the best constant (test set): {msle_const}')

MSLE for the best constant (test set): 0.42757520337601745


Now, let's implement a standard linear regression model. 

In [28]:
def generate_sets(train, test):

  train_y = train['cena'].to_numpy()
  test_y = test['cena'].to_numpy()

  train_x = train.drop('cena', axis=1)
  test_x = test.drop('cena', axis=1)

  mean = train_x.mean().to_numpy()
  std = train_x.std().to_numpy()

  # scaling and centralizing 
  train_x = (train_x - mean) / std
  # adding bias
  train_x = np.hstack((np.ones_like(train_x), train_x))


  # scaling and centralizing 
  test_x = (test_x - mean) / std
  # adding bias
  test_x = np.hstack((np.ones_like(test_x), test_x))

  return train_x, train_y, test_x, test_y

  
train_x, train_y, test_x, test_y = generate_sets(train, test)


In [33]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################

lr = 0.001

n = len(train_y)

def evaluate(ws, xs, ys):
  ps = np.dot(xs, ws)
  assert len(ys) == len(ps)
  return np.mean((ys - ps)**2)

def fit(x, y, steps):
  W = np.random.normal(size=x.shape[1])
  for i in range(steps):
    predictions = np.dot(x, W)
    delta = predictions - y

    gradient = (2/n) * np.dot(x.T, delta)
    W -= lr * gradient 

    if i % 10000 == 0:
      loss = evaluate(W, x, y)
      print('Iter: {:>3} Loss: {:8.8f}'.format(i, loss))
  return W

In [34]:
W = fit(train_x, train_y, 100000)
pred = test_x @ W
print(f'Score after regression: {msle(test_y, pred)}')

Iter:   0 Loss: 321882285823.56243896
Iter: 10000 Loss: 4158719270.13698196
Iter: 20000 Loss: 3910094294.88643456
Iter: 30000 Loss: 3874334255.56027508
Iter: 40000 Loss: 3869190051.81722069
Iter: 50000 Loss: 3868450040.45950174
Iter: 60000 Loss: 3868343587.28677607
Iter: 70000 Loss: 3868328273.63264656
Iter: 80000 Loss: 3868326070.71095943
Iter: 90000 Loss: 3868325753.81311750


Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

In [38]:
W_log = fit(train_x, np.log(1 + train_y), 100000)
pred_log = np.expm1(test_x @ W_log)
print(f'Score after regression: {msle(test_y, pred_log)}')

Iter:   0 Loss: 150.01038819
Iter: 10000 Loss: 0.03186233
Iter: 20000 Loss: 0.03047173
Iter: 30000 Loss: 0.03027889
Iter: 40000 Loss: 0.03025115
Iter: 50000 Loss: 0.03024716
Iter: 60000 Loss: 0.03024659
Iter: 70000 Loss: 0.03024651
Iter: 80000 Loss: 0.03024650
Iter: 90000 Loss: 0.03024649
Score after regression: 0.03656905572854784


Hint: 
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint: 
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2: 
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [43]:
###############################################
# TODO: Implement the feature engieering part #
###############################################

columns = [x for x in train.columns if x != 'cena']

def multiply_columns(df):
  a,b = df.shape
  new_df = np.zeros(shape=(a, int(b*(b+1)/2)))
  k=0
  for i in range(b):
    for j in range(i, b):
      new_df[:, k] = df[:, i] * df[:, j]
      k += 1
  return new_df 

train_x_new = multiply_columns(train_x)
test_x_new = multiply_columns(test_x)

In [44]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################

W_log = fit(train_x_new, np.log(1+train_y), 100000)
pred = np.expm1(test_x_new @ W_log)
print(f'Score after regression = {msle(test_y, pred)}')

Iter:   0 Loss: 480.56162054
Iter: 10000 Loss: 0.03354522
Iter: 20000 Loss: 0.01146281
Iter: 30000 Loss: 0.00808885
Iter: 40000 Loss: 0.00743913
Iter: 50000 Loss: 0.00726251
Iter: 60000 Loss: 0.00718218
Iter: 70000 Loss: 0.00712589
Iter: 80000 Loss: 0.00707783
Iter: 90000 Loss: 0.00703402
Score after regression = 0.01895463477053715
